In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import malaya_speech
import malaya_speech.config
import malaya_speech.train.model.unet_enhancement as unet
from malaya_speech.train.model import enhancement
import tensorflow as tf






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





In [4]:
x = tf.placeholder(tf.float32, (None, 1))
partitioned_x = malaya_speech.tf_featurization.pad_and_partition(x, 16384)
model = unet.Model(partitioned_x)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Tensor("strided_slice_3:0", shape=(?, 8192, 24), dtype=float32)
Tensor("strided_slice_4:0", shape=(?, 4096, 48), dtype=float32)
Tensor("strided_slice_5:0", shape=(?, 2048, 72), dtype=float32)
Tensor("strided_slice_6:0", shape=(?, 1024, 96), dtype=float32)
Tensor("strided_slice_7:0", shape=(?, 512, 120), dtype=float32)
Tensor("strided_slice_8:0", shape=(?, 256, 144), dtype=float32)
Tensor("strided_slice_9:0", shape=(?, 128, 168), dtype=float32)
Tensor("strided_slice_10:0", shape=(?, 64, 192), dtype=float32)
Tensor("strided_slice_11:0", shape=(?, 32, 216), dtype=float32)
Tensor("strided_slice_12:0", shape=(?, 16, 240), dtype=float32)
Tensor("strided_slice_13:0", shape=(?, 8, 264), dtype=float32)
Tensor("strided_slice_14:0", shape=(?, 4, 288), dtype=float32)
Tensor("sequential_12/leaky_re_lu_12/LeakyRelu:0", shape=(?, 4, 288), dtype=float32)
Tensor("sequential_13/leaky_re_lu_13/LeakyRelu:0", shape=(?, 8, 288)

In [5]:
model.logits

<tf.Tensor 'sequential_25/activation_6/Tanh:0' shape=(?, 16384, 1) dtype=float32>

In [6]:
logits = tf.reshape(model.logits, (-1, 1))
logits = logits[:tf.shape(x)[0]]
logits

<tf.Tensor 'strided_slice_40:0' shape=(?, 1) dtype=float32>

In [7]:
snr = enhancement.loss.snr(logits, x)
sdr = enhancement.loss.sdr(logits, x)

In [8]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [9]:
import numpy as np

In [10]:
y, sr = malaya_speech.load('../speech/record/savewav_2020-09-06_21-48-43_174842.wav', sr = 22050)

In [11]:
y_, snr_, sdr_ = sess.run([model.logits, snr, sdr], feed_dict = {x: np.expand_dims(y, -1)})
y_.shape

(16, 16384, 1)

In [12]:
import IPython.display as ipd

ipd.Audio(y_.reshape((-1)), rate = 22050)

In [13]:
snr_, sdr_

(-15.438549, -3.0989852e-08)